# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (604, 26)
y_train: (604, 2)
x_test: (604, 26)
y_test: (604, 2)


# Pass inputs through stacked autoencoder

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-classify-sae-04/'

autoencoder_1_path = 'results/34-autoencoder-1-01/model/2018-03-09-03-01-07/weights.hdf5'
autoencoder_2_path = 'results/37-autoencoder-2-01/model/2018-03-09-05-06-53/weights.hdf5'
weights_1 = load_model(autoencoder_1_path)
weights_2 = load_model(autoencoder_2_path)

weights_1.summary()
weights_2.summary()

# x_train_ae = weights_1.predict(x_train)
# x_train_ae = weights_2.predict(x_train_ae)

# x_test_ae = weights_1.predict(x_test)
# x_test_ae = weights_2.predict(x_test_ae)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                270       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 26)                286       
Total params: 556
Trainable params: 556
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_2 (Den

In [5]:
x = Input(shape=(x_train.shape[1],))    
y = Dense(units = 10, activation='relu')(x)
y = Dense(units = 8, activation='relu')(y)
model = Model(x, y)

In [6]:
model.layers[1].set_weights(weights_1.get_layer('dense_9').get_weights())
model.layers[2].set_weights(weights_2.get_layer('dense_2').get_weights())

In [7]:
x_train_ae = model.predict(x_train)
x_test_ae = model.predict(x_test)

# KNN

In [8]:
knn = KNeighborsClassifier()
knn.fit(x_train_ae, y_train)
y_pred = knn.predict(x_test_ae)
accuracy_score(y_test, y_pred)

0.6556291390728477